# Introduction

## Goal
The goal of this lab is to familiarize yourself with Particle Swarm Optimization and study the effect of parametrization on the algorithmic performance.

Note once again that, unless otherwise specified, in this module's exercises we will use real-valued genotypes and that the aim of the algorithms will be to *minimize* the fitness function $f(\mathbf{x})$, i.e. lower values correspond to a better fitness!

## Exercise 1

As a first exercise, we will run a simple 2D Boids simulator, based on the Reynolds' flocking rules we have seen during the lectures. Although this exercise is not strictly related to PSO, it provides a good source of inspiration (and intuition) on how PSO works. 

The simulator allows you to change various aspects of the simulation, specifically the total number of boids, the number of neighbors whose information is collected by each boid (to determine cohesion, alignment, and separation), and the relative weights of each of the 3 flocking rules (behavior coefficients). Spend some time with the simulator, and try different simulation configurations.

To help you figure out the behavior of the boids, you can find below the implementation of the `boid` class extracted from the source code of the simulator. In particular, check the `update` method.

- What is the effect of each behavior coefficient?

Cohesion: This rule encourages each boid to move towards the center of mass of its neighboring boids. When the cohesion coefficient is higher, the boids are more likely to group together and move as a cohesive unit, almost like they are attracted to stay close to one another.

Alignment: This rule causes the boids to align their direction of movement with the average direction of their nearby boids. The higher the alignment coefficient, the more closely the boids will match the direction of their neighbors, which helps them stay coordinated and move in the same direction, preventing them from scattering in different ways.

Separation: This rule ensures that the boids don’t get too close to one another, maintaining a safe distance to avoid crowding or collisions. A higher separation coefficient increases their tendency to keep space between each other, which prevents them from clustering too tightly or crashing into one another.

- Which combination of coefficients leads to the most ``natural'' flock behavior? 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from utils.utils_06.main import run
from utils.utils_06.rules import Rules
import os

class Boid(Rules):
    def __init__(self, screen_width, screen_height):
        super().__init__(screen_width, screen_height)
        self.position = Vector2(random.uniform(0, screen_width), random.uniform(0, screen_height))
        self.velocity = Vector2(random.uniform(-1, 1), random.uniform(-1, 1))
        self.radius = 100

    # Draw the boid
    def draw(self, screen):
        pg.draw.circle(screen,'red', (int(self.position.x), int(self.position.y)), 5)

    ### Update the position of the boid
    def update(self, boids, ALIGNMENT, COHESION, SEPARATION):
        
        ### Weights of the rules

        ### Neighbors in range
        n = Rules.neighbors(self, boids)

        ### Rules to follow
        alignment = ALIGNMENT * Rules.match_velocity(self, n)
        cohesion = COHESION * Rules.fly_towards_center(self, n)
        separation_from_boids = SEPARATION * Rules.keep_distance_away(self, n, 9)
        
        # Update velocity 
        self.velocity += alignment + cohesion + separation_from_boids

        # Limit the speed of the boids
        self.velocity.scale_to_length(5)

        # Update position
        self.position += self.velocity
        
        # Wrap the position of the boids
        Rules.bound_position(self)

        
"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""


runs = {
    'base': [50,    0.5,    0.5,    0.5],
    'alligment more': [50,    1,    0.5,    0.5],
    'cohesion more': [50,    0.5,    1,    0.5],
    'separation more': [50,    0.5,    0.5,    1],
    'boids more': [100,    0.5,    0.5,    0.5],
    #'1,1.5,1': [50,    1,    1.5,    1],
    #'1,1.5,0.5': [50,    1,    1.5,    0.5],
    '1,0.5,1': [50,    1,    0.5,    1],

}
for run in runs:
    print(run, runs[run])
    num_boids = runs[run][0]   # advice: for graphical reasons avoid to use num_boids > 400
    alignment = runs[run][1]  #default: 0.5
    cohesion = runs[run][2]   # default: 0.5
    separation = runs[run][3] # default: 0.5
    title=run
    # make sure you are calling the right version of python in the process below
   
    os.popen(f'python utils/utils_06/main.py {num_boids} {alignment} {cohesion} {separation} {title}').read()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
1,1.5,1 [50, 1, 1.5, 1]
1,1.5,1
1,1.5,0.5 [50, 1, 1.5, 0.5]
1,1.5,0.5
1,0.5,1 [50, 1, 0.5, 1]
1,0.5,1


: 

## Exercise 2

In this exercise we will perform a comparative analysis of the results of Genetic Algorithm (as seen in Lab 2), Evolution Strategies (as seen in Lab 3) and Particle Swarm Optimization. 
The script will perform a single run of GA, ES and PSO, on one of the benchmark functions we have seen in the previous labs. As usual, the algorithm parametrization is shown in the code and can be easily modified.

Questions:
-  What kind of behavior does PSO have on different benchmark functions (change the parameter `args["problem_class"]` to try at least a couple of functions), in comparison with the EAs? Does it show better or worse results? Does it converge faster or not?

For some functions, PSO may be more efficient due to its nature of exploring the search space through particle velocities rather than relying on mutation/crossover strategies.
PSO converges more quickly on simpler functions (like Sphere) but might struggle more with complex multimodal functions (like Rastrigin or Ackley) compared to GA and ES, depending on the tuning of the parameters.

- What happens if you run the script multiple times? Do the various algorithms (and especially PSO) show consistent behavior?

- Increase the problem dimensionality (`num_vars`, by default set to 2), e.g. to 10 or more. What do you observe in this case?

Increasing the dimensionality of the problem makes optimization significantly harder for all algorithms, but the effect is more pronounced for GA and ES. PSO generally shows better robustness when dealing with higher-dimensional problems, though its performance also deteriorates, particularly on complex functions like Schwefel and Ackley. In high-dimensional spaces, it’s crucial to fine-tune the algorithms and possibly use more sophisticated approaches to prevent them from getting stuck in local optima.

-  Change the population size (by changing`args["pop_size"]`, by default set to 50) and the number of generations (by changing `args["max_generations"]`, by default set to 100), such that their product is fixed (e.g. $25 \times 200$, $100 \times 50$, etc.). Try two or three different combinations and observe the behavior of the three different algorithms. What do you observe in this case? Is it better to have smaller/larger populations or a smaller/larger number of generations? Why?

Impact of More Generations (Smaller Population):

Across all functions, more generations with a smaller population (25 × 200) generally yield better results, particularly for PSO and ES. The added time for exploration allows the algorithms to refine their solutions and avoid premature convergence, especially on more complex landscapes like Ackley and Rosenbrock.
PSO benefits the most from this configuration, improving significantly on Ackley and Rosenbrock. This indicates that PSO’s swarm-based exploration thrives when it has more generations to explore the search space, even with a smaller population.
Impact of Larger Population (Fewer Generations):

Larger populations with fewer generations (100 × 50) tend to produce worse results for all three algorithms. The algorithms are likely able to explore the search space more broadly at first, but the limited number of generations prevents them from fine-tuning and improving the solutions over time. This is especially true for ES and PSO, which perform particularly poorly on Rosenbrock and Ackley with this configuration.
GA also struggles in this configuration, showing that more generations are critical for its crossover and mutation mechanisms to effectively improve solutions.
Algorithm-Specific Trends:

GA: Tends to perform better with more generations, as it relies on evolutionary processes that take time to fine-tune solutions. GA suffers the most when generations are limited (e.g., 100 × 50).
ES: Performs well with more generations and suffers heavily with fewer generations, particularly on complex landscapes like Rosenbrock and Schwefel. It often struggles to escape local optima when generations are limited.
PSO: Shows significant improvement when given more generations, suggesting that its exploration-exploitation balance is more effective when allowed to run for a longer period. It performs the worst with fewer generations, as it cannot fully explore the search space.



#### multiple runs

In [9]:
# -*- coding: utf-8 -*-

from pylab import *
import sys
from inspyred import ec
from inspyred import benchmarks


from utils.utils_06.inspyred_utils import NumpyRandomWrapper
import utils.utils_06.ga as ga
import utils.utils_06.es as es
import utils.utils_06.pso as pso
"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""
runs = {
    'Sphere': benchmarks.Sphere,
    'Rosenbrock': benchmarks.Rosenbrock,
    'Ackley': benchmarks.Ackley,
    'Schwefel': benchmarks.Schwefel

}

num_repeats = 10  # Number of times to repeat each configuration
ga_fitnesses = []
es_fitnesses = []
pso_fitnesses = []

args = {}
num_vars = 2 # Number of dimensions of the search space
# common parameters
args["max_generations"] = 100 # Number of generations
args["pop_size"] = 50 # population size
# parameters for the GA
args["gaussian_stdev"] = 0.5 # Standard deviation of the Gaussian mutations
args["mutation_rate"] = 0.5 # fraction of loci to perform mutation on
args["tournament_size"] = 2
args["num_elites"] = 1 # number of elite individuals to maintain in each gen
# parameters for the ES
args["num_offspring"] = 100 #lambda
args["sigma"] = 1.0 # default standard deviation
args["strategy_mode"] = es.INDIVIDUAL # es.GLOBAL, es.INDIVIDUAL, None
args["mixing_number"] = 1 #rho
# parameters for the PSO
args["topology"] = pso.STAR #pso.RING, pso.STAR
args["neighborhood_size"] = 5   #used only for the ring topology
args["inertia"] = 0.5
args["cognitive_rate"] = 2.1
args["social_rate"] = 2.1


for run in runs:
    # Reset fitness results for the current configuration
    ga_fitnesses.clear()
    es_fitnesses.clear()
    pso_fitnesses.clear()

    # the problem class
    args["problem_class"] = runs[run]

    for i in range(num_repeats):

        display = False # Plot initial and final populations

        rng = NumpyRandomWrapper()
        # Run GA
        args["fig_title"] = "GA"
        best_individual, best_fitness, final_pop = ga.run_ga(rng,num_vars=num_vars,
                                                display=display,use_log_scale=True,
                                                **args)
        ga_fitnesses.append(best_fitness)
        #print(f"Best GA fitness in run {i+1}: {best_fitness}")
        # Run ES
        args["fig_title"] = "ES"
        best_individual, best_fitness, final_pop = es.run_es(rng,num_vars=num_vars,
                                                    display=display,use_log_scale=True,
                                                    **args)
        es_fitnesses.append(best_fitness)
        #print(f"Best ES fitness in run {i+1}: {best_fitness}")
        # Run PSO
        args["fig_title"] = "PSO"
        best_individual, best_fitness, final_pop = pso.run_pso(rng,num_vars=num_vars,
                                                    display=display,use_log_scale=True,
                                                    **args)
        pso_fitnesses.append(best_fitness)
        #print(f"Best PS= fitness in run {i+1}: {best_fitness}")

        ioff()
        show()
    # Calculate and display mean and variance for each algorithm
    print(f"\n\n--- Statistics for {run} ---")
    print(f"GA:    Mean = {np.mean(ga_fitnesses):.7f}, Standard Deviation = {np.std(ga_fitnesses):.7f}")
    print(f"ES:    Mean = {np.mean(es_fitnesses):.7f}, Standard Deviation = {np.std(es_fitnesses):.7f}")
    print(f"PSO:   Mean = {np.mean(pso_fitnesses):.7f}, Standard Deviation = {np.std(pso_fitnesses):.7f}")



--- Statistics for Sphere ---
GA:    Mean = 0.0000139, Standard Deviation = 0.0000118
ES:    Mean = 0.0000000, Standard Deviation = 0.0000000
PSO:   Mean = 0.0000000, Standard Deviation = 0.0000000


--- Statistics for Rosenbrock ---
GA:    Mean = 0.0021399, Standard Deviation = 0.0018857
ES:    Mean = 0.0113013, Standard Deviation = 0.0301609
PSO:   Mean = 0.0002662, Standard Deviation = 0.0003608


--- Statistics for Ackley ---
GA:    Mean = 0.0078774, Standard Deviation = 0.0050192
ES:    Mean = 0.0000005, Standard Deviation = 0.0000003
PSO:   Mean = 0.0000006, Standard Deviation = 0.0000005


--- Statistics for Schwefel ---
GA:    Mean = 0.0997073, Standard Deviation = 0.2989936
ES:    Mean = 57.2454928, Standard Deviation = 75.1412790
PSO:   Mean = 23.6879176, Standard Deviation = 47.3752213


#### problem dimensionality

In [6]:
# -*- coding: utf-8 -*-

from pylab import *
import sys
from inspyred import ec
from inspyred import benchmarks


from utils.utils_06.inspyred_utils import NumpyRandomWrapper
import utils.utils_06.ga as ga
import utils.utils_06.es as es
import utils.utils_06.pso as pso
"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""
runs = {
    'num_vars basic Sphere': [benchmarks.Sphere, 2],
    'num_vars more Sphere': [benchmarks.Sphere, 10],
    'num_vars basic Rosenbrock': [benchmarks.Rosenbrock, 2],
    'num_vars more Rosenbrock': [benchmarks.Rosenbrock, 10],
    'num_vars basic Ackley': [benchmarks.Ackley, 2],
    'num_vars more Ackley': [benchmarks.Ackley, 10],
    'num_vars basic Schwefel': [benchmarks.Schwefel, 2],
    'num_vars more Schwefel': [benchmarks.Schwefel, 10],

}


args = {}

# common parameters
args["max_generations"] = 100 # Number of generations
args["pop_size"] = 50 # population size
# parameters for the GA
args["gaussian_stdev"] = 0.5 # Standard deviation of the Gaussian mutations
args["mutation_rate"] = 0.5 # fraction of loci to perform mutation on
args["tournament_size"] = 2
args["num_elites"] = 1 # number of elite individuals to maintain in each gen
# parameters for the ES
args["num_offspring"] = 100 #lambda
args["sigma"] = 1.0 # default standard deviation
args["strategy_mode"] = es.INDIVIDUAL # es.GLOBAL, es.INDIVIDUAL, None
args["mixing_number"] = 1 #rho
# parameters for the PSO
args["topology"] = pso.RING #pso.RING, pso.STAR
args["neighborhood_size"] = 5   #used only for the ring topology
args["inertia"] = 0.5
args["cognitive_rate"] = 2.1
args["social_rate"] = 2.1


for run in runs:
    
    print()
    print('----------------------------------------------------')
    print()
    print(run, runs[run])

    # the problem class
    args["problem_class"] = runs[run][0]
    num_vars = runs[run][1] # Number of dimensions of the search space

    display = False # Plot initial and final populations

    rng = NumpyRandomWrapper()
    # Run GA
    args["fig_title"] = "GA"
    best_individual, best_fitness, final_pop = ga.run_ga(rng,num_vars=num_vars,
                                            display=display,use_log_scale=True,
                                            **args)
    ga_fitnesses.append(best_fitness)
    print(f"Best GA fitness : {best_fitness}")
    # Run ES
    args["fig_title"] = "ES"
    best_individual, best_fitness, final_pop = es.run_es(rng,num_vars=num_vars,
                                                display=display,use_log_scale=True,
                                                **args)
    es_fitnesses.append(best_fitness)
    print(f"Best ES fitness : {best_fitness}")
    # Run PSO
    args["fig_title"] = "PSO"
    best_individual, best_fitness, final_pop = pso.run_pso(rng,num_vars=num_vars,
                                                display=display,use_log_scale=True,
                                                **args)
    pso_fitnesses.append(best_fitness)
    print(f"Best PS fitness: {best_fitness}")

    ioff()
    show()



----------------------------------------------------

num_vars basic Sphere [<class 'inspyred.benchmarks.Sphere'>, 2]
Best GA fitness : 5.102534098785902e-06
Best ES fitness : 3.418557050742053e-15
Best PS fitness: 4.515795860184033e-15

----------------------------------------------------

num_vars more Sphere [<class 'inspyred.benchmarks.Sphere'>, 10]
Best GA fitness : 0.39668095049764274
Best ES fitness : 0.02572489055550019
Best PS fitness: 0.059650839821255776

----------------------------------------------------

num_vars basic Rosenbrock [<class 'inspyred.benchmarks.Rosenbrock'>, 2]
Best GA fitness : 0.000774842023694006
Best ES fitness : 0.04234872146363992
Best PS fitness: 4.472513548816972e-05

----------------------------------------------------

num_vars more Rosenbrock [<class 'inspyred.benchmarks.Rosenbrock'>, 10]
Best GA fitness : 31.75568451827765
Best ES fitness : 28.109154387913993
Best PS fitness: 203.51909688058743

-------------------------------------------------

#### cambio parametri

In [8]:
from pylab import *
import sys
from inspyred import ec
from inspyred import benchmarks

from utils.utils_06.inspyred_utils import NumpyRandomWrapper
import utils.utils_06.ga as ga
import utils.utils_06.es as es
import utils.utils_06.pso as pso

"""
------------------------------------------------------------------------
Edit this part to do the exercises
------------------------------------------------------------------------
"""
# New structure for 'runs'
runs = {
    'run1': {"function": benchmarks.Sphere, "num_vars": 10, "pop_size": 50, "max_generations": 100},
    'run2': {"function": benchmarks.Sphere, "num_vars": 10, "pop_size": 25, "max_generations": 200},
    'run3': {"function": benchmarks.Sphere, "num_vars": 10, "pop_size": 100, "max_generations": 50},
    'run4': {"function": benchmarks.Rosenbrock, "num_vars": 10, "pop_size": 50, "max_generations": 100},
    'run5': {"function": benchmarks.Rosenbrock, "num_vars": 10, "pop_size": 25, "max_generations": 200},
    'run6': {"function": benchmarks.Rosenbrock, "num_vars": 10, "pop_size": 100, "max_generations": 50},
    'run7': {"function": benchmarks.Ackley, "num_vars": 10, "pop_size": 50, "max_generations": 100},
    'run8': {"function": benchmarks.Ackley, "num_vars": 10, "pop_size": 25, "max_generations": 200},
    'run9': {"function": benchmarks.Ackley, "num_vars": 10, "pop_size": 100, "max_generations": 50},
    'run10': {"function": benchmarks.Schwefel, "num_vars": 10, "pop_size": 50, "max_generations": 100},
    'run11': {"function": benchmarks.Schwefel, "num_vars": 10, "pop_size": 25, "max_generations": 200},
    'run12': {"function": benchmarks.Schwefel, "num_vars": 10, "pop_size": 100, "max_generations": 50},
}

# Parameters for GA, ES, and PSO are still defined globally
args = {}

# parameters for the GA
args["gaussian_stdev"] = 0.5  # Standard deviation of the Gaussian mutations
args["mutation_rate"] = 0.5   # fraction of loci to perform mutation on
args["tournament_size"] = 2
args["num_elites"] = 1        # number of elite individuals to maintain in each gen

# parameters for the ES
args["num_offspring"] = 100   # lambda (offspring)
args["sigma"] = 1.0           # default standard deviation
args["strategy_mode"] = es.INDIVIDUAL  # es.GLOBAL, es.INDIVIDUAL, None
args["mixing_number"] = 1     # rho

# parameters for the PSO
args["topology"] = pso.RING   # pso.RING, pso.STAR
args["neighborhood_size"] = 5  # used only for the ring topology
args["inertia"] = 0.5
args["cognitive_rate"] = 2.1
args["social_rate"] = 2.1

# To store results for each combination
ga_fitnesses = []
es_fitnesses = []
pso_fitnesses = []

# Loop through different configurations
for run_name, run_params in runs.items():
    print(f"\n\nRunning: {run_name} with {run_params}")

    # Extract parameters from the dictionary
    args["problem_class"] = run_params["function"]
    num_vars = run_params["num_vars"]  # Number of dimensions of the search space
    args["pop_size"] = run_params["pop_size"]  # Population size
    args["max_generations"] = run_params["max_generations"]  # Generations

    display = False  # Plot initial and final populations
    rng = NumpyRandomWrapper()

    # Run GA
    args["fig_title"] = "GA"
    best_individual, best_fitness, final_pop = ga.run_ga(rng, num_vars=num_vars,
                                                        display=display, use_log_scale=True,
                                                        **args)
    ga_fitnesses.append(best_fitness)
    print(f"Best GA fitness : {best_fitness}")

    # Run ES
    args["fig_title"] = "ES"
    best_individual, best_fitness, final_pop = es.run_es(rng, num_vars=num_vars,
                                                        display=display, use_log_scale=True,
                                                        **args)
    es_fitnesses.append(best_fitness)
    print(f"Best ES fitness : {best_fitness}")

    # Run PSO
    args["fig_title"] = "PSO"
    best_individual, best_fitness, final_pop = pso.run_pso(rng, num_vars=num_vars,
                                                          display=display, use_log_scale=True,
                                                          **args)
    pso_fitnesses.append(best_fitness)
    print(f"Best PSO fitness: {best_fitness}")

# Show results
ioff()
show()



Running: run1 with {'function': <class 'inspyred.benchmarks.Sphere'>, 'num_vars': 10, 'pop_size': 50, 'max_generations': 100}
Best GA fitness : 0.24868964404768795
Best ES fitness : 0.000276720343761519
Best PSO fitness: 0.006474592122361082


Running: run2 with {'function': <class 'inspyred.benchmarks.Sphere'>, 'num_vars': 10, 'pop_size': 25, 'max_generations': 200}
Best GA fitness : 0.1377845925897828
Best ES fitness : 9.717170608445408e-06
Best PSO fitness: 0.00021661924735165674


Running: run3 with {'function': <class 'inspyred.benchmarks.Sphere'>, 'num_vars': 10, 'pop_size': 100, 'max_generations': 50}
Best GA fitness : 0.36746196689187544
Best ES fitness : 1.2824281343874615
Best PSO fitness: 0.29196956749594777


Running: run4 with {'function': <class 'inspyred.benchmarks.Rosenbrock'>, 'num_vars': 10, 'pop_size': 50, 'max_generations': 100}
Best GA fitness : 157.89298696579198
Best ES fitness : 69.17224973426129
Best PSO fitness: 97.94475136197103


Running: run5 with {'funct

## Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 

- When do you think it is useful to have a lower (higher) cognitive learning rate? What about the social learning rate?
- From a biological point of view, which neighborhood topology do you consider as the most plausible?
